# Model comparisons

In [ ]:
from face_alignment import FaceAlignment, LandmarksType

from loreal_poc.dataloaders.loaders import DataLoaderFFHQ
from loreal_poc.dataloaders.wrappers import (
    CroppedDataLoader,
    ResizedDataLoader,
    ColoredDataLoader,
    BlurredDataLoader,
    FilteredDataLoader,
    HeadPoseDataLoader,
    EthnicityDataLoader,
    CachedDataLoader,
)

from loreal_poc.models.wrappers import OpenCVWrapper, FaceAlignmentWrapper
from loreal_poc.tests.performance import NMEMean
from loreal_poc.tests.base import Test, TestDiff
from loreal_poc.marks.facial_parts import FacialParts

### Loading FFHQ dataset

In [ ]:
dl_ref = DataLoaderFFHQ("ffhq")

### Loading landmark-detection models
- FaceAlignment
- OpenCV

In [ ]:
models = {
    "FaceAlignment": FaceAlignmentWrapper(model=FaceAlignment(LandmarksType.TWO_D, device="cpu", flip_input=False)),
    "OpenCV": OpenCVWrapper(),
}
models.pop("FaceAlignment")

In [ ]:
results = []

### Case 1: Cropped Images

In [ ]:
facial_parts = [FacialParts.LEFT_HALF.value, FacialParts.RIGHT_HALF.value]

for model in models.values():
    for fp in facial_parts:
        dl = CroppedDataLoader(dl_ref, part=fp)
        results.append(
            TestDiff(metric=NMEMean, threshold=1)
            .run(
                model=model,
                dataloader=dl,
                dataloader_ref=dl_ref,
                facial_part=fp,
            )
            .to_dict()
        )

### Case 2A: Resized Images

In [ ]:
for model in models.values():
    dl = ResizedDataLoader(dl_ref, scales=0.5)
    results.append(
        TestDiff(metric=NMEMean, threshold=1)
        .run(
            model=model,
            dataloader=dl,
            dataloader_ref=dl_ref,
        )
        .to_dict()
    )

### Case 2B: Recolored Images

In [ ]:
for model in models.values():
    transformed_dl = ColoredDataLoader(dl)
    results.append(
        TestDiff(metric=NMEMean, threshold=1)
        .run(
            model=model,
            dataloader=transformed_dl,
            dataloader_ref=dl,
        )
        .to_dict()
    )

### Case 2C: Blurred Images

In [ ]:
for model in models.values():
    dl = BlurredDataLoader(dl_ref)
    results.append(
        TestDiff(metric=NMEMean, threshold=1)
        .run(
            model=model,
            dataloader=dl,
            dataloader_ref=dl_ref,
        )
        .to_dict()
    )

### Case 3: Head Pose

In [ ]:
cached_dl = CachedDataLoader(HeadPoseDataLoader(dl_ref), cache_size=None, cache_img=False, cache_marks=False)

head_poses = [lambda elt: elt[2]["headPose"]["roll"] > 0, lambda elt: elt[2]["headPose"]["roll"] < 0]

for model in models.values():
    for hp in head_poses:
        dl = FilteredDataLoader(cached_dl, hp)
        results.append(
            TestDiff(metric=NMEMean, threshold=1)
            .run(
                model=model,
                dataloader=dl,
                dataloader_ref=dl_ref,
            )
            .to_dict()
        )

### Case 4: Ethnicity

In [ ]:
ethnicity_dl = EthnicityDataLoader(dl_ref, ethnicity_map={"indian": "asian"})
cached_dl = CachedDataLoader(ethnicity_dl, cache_size=None, cache_img=False, cache_marks=False)

ethnicities = [lambda elt: elt[2]["ethnicity"] == "asian"]

for model in models.values():
    for e in ethnicities:
        dl = FilteredDataLoader(cached_dl, e)
        results.append(
            TestDiff(metric=NMEMean, threshold=1)
            .run(
                model=model,
                dataloader=dl,
                dataloader_ref=dl_ref,
            )
            .to_dict()
        )

In [ ]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
report = pd.DataFrame(results)
report.groupby(["model_name"]).apply(display)